In [1]:
import ROOT as r

Welcome to JupyROOT 6.24/06


In [30]:
f=r.TFile("/data/maclwong/Ben_Bkg_Samples/v2/user.bhodkins.700401.Znunugamma.mc20e.v2.0_ANALYSIS.root/user.bhodkins.42165174._000001.ANALYSIS.root","RO")
t=f.Get("analysis")

passing=0
for e in t:

    met=e.met_met_NOSYS
    met_phi=e.met_phi_NOSYS
    met_tlv=r.TLorentzVector()
    met_tlv.SetPtEtaPhiM(met,0,met_phi,0)
    
    if e.met_met_NOSYS<250000: continue
    
    nelectrons = len(e.el_pt_NOSYS)
    nmuons     = len(e.mu_pt_NOSYS)
    if (nelectrons+nmuons)>0: continue
    
    jets=[]
    mindphijetsmet=999
    btags=0
    for j in range(len(e.jet_pt_NOSYS)):
        tlv=r.TLorentzVector()
        tlv.SetPtEtaPhiM(e.jet_pt_NOSYS[j],
                         e.jet_eta[j],
                         e.jet_phi[j],
                         0)
        jets.append(tlv)
        mindphijetsmet=min(mindphijetsmet,tlv.DeltaPhi(met_tlv))
        if ord(e.jet_btag_select[j])!=0: btags+=1
    if jets[0].Pt()<100000: continue
    if mindphijetsmet<0.4: continue
    if btags>0: continue

    photons=[]
    mindrphjet=999
    for p in range(len(e.ph_pt_NOSYS)):
        tlv=r.TLorentzVector()
        tlv.SetPtEtaPhiM(e.ph_pt_NOSYS[p],
                         e.ph_eta[p],
                         e.ph_phi[p],
                         0)
        photons.append(tlv)     
        for j in jets:
            mindrphjet=min(mindrphjet,tlv.DeltaPhi(j))
    if mindrphjet<0.4: continue
    
    if photons[0].Pt()<10000: continue

    if not e.in_vgamma_overlap_7: continue

    
    passing+=1
print(passing)

1147


Error in <THashList::Delete>: A list is accessing an object (0x33e7e1e0) already deleted (list name = THashList)
